In [1]:
import os
import numpy as np
import pandas as pd

import riskflow_jupyter

In [2]:
rundate = '2022-07-07'
if os.name=='nt':
    path = os.path.join('U:\\CVA_JSON', rundate)
    path_transform={
        '\\\\ICMJHBMVDROPPRD\\AdaptiveAnalytics\\Inbound\\MarketData':
        '\\\\ICMJHBMVDROPUAT\\AdaptiveAnalytics\\Inbound\\MarketData'}
else:
    path = os.path.join('/media/vretiel/3EFA4BCDFA4B7FDF/Media/Data/crstal/CVA_JSON', rundate)
    path_transform={
            '//ICMJHBMVDROPPRD/AdaptiveAnalytics/Inbound/MarketData': 
            '/media/vretiel/3EFA4BCDFA4B7FDF/Media/Data/crstal/CVA_JSON'}
    
cx = riskflow_jupyter.rf.Context(
    path_transform=path_transform,
    file_transform={
        'CVAMarketData_Calibrated.dat': 'CVAMarketData_Calibrated_New.json',
        'MarketData.dat': 'MarketData.json'
    })

In [3]:
from riskflow_widgets import Table, Three, to_json

In [4]:
# cx.load_json(os.path.join(path, 'InputAAJ_CrB_Russellstone_Treasury_ISDA.json'))
cx.load_json(os.path.join(path, 'InputAAJ_CrB_BNP_Paribas__Paris__ISDA.json'))


In [ ]:
import ipywidgets as widgets


In [ ]:
factor = riskflow_jupyter.rf.utils.Factor('FXVol',('GBP','ZAR'))
# factor = riskflow_jupyter.rf.utils.Factor('InterestYieldVol',('CAD_SMILE_ICE',))

vol = riskflow_jupyter.rf.riskfactors.construct_factor( factor,
    cx.current_cfg.params['Price Factors'],
    cx.current_cfg.params['Price Factor Interpolation']
)

In [ ]:
j=riskflow_jupyter.load_table_from_vol(vol)
#to_json(j)
#np.isnan(None)

In [ ]:
# g = np.vstack([x[0], heights]).tolist()
t = Three(description='Test')
t.observe(lambda x:x['new'], 'value')
t.value = to_json(j)

In [ ]:
t

In [ ]:
%debug

In [ ]:
data

In [ ]:
t.value

In [ ]:
plt_text1.visible=False

In [ ]:
from ipyfilechooser import FileChooser

# Create and display a FileChooser widget
fc = FileChooser('/home/vretiel/miniconda3/')
display(fc)

In [ ]:
cx

In [5]:
pp = riskflow_jupyter.PortfolioPage(cx)

In [ ]:
pp.main_container

In [ ]:
rfp = riskflow_jupyter.RiskFactorsPage(cx)

In [ ]:
rfp.main_container


In [6]:
import importlib
importlib.reload(riskflow_jupyter)

<module 'riskflow_jupyter' from 'C:\\Users\\shuaib.osman\\PycharmProjects\\riskflow\\riskflow_jupyter.py'>

In [7]:
cp = riskflow_jupyter.CalculationPage(cx)
cp.main_container

dict_keys(['frames', 'calc', 'output'])


{'Base_Date': Timestamp('2022-07-07 00:00:00'), 'Currency': 'EUR', 'Base_time_grid': '0d 2d 1w(1w) 3m(1m) 2y(3m)', 'Simulation_Batches': 1, 'Batch_Size': 1024, 'Random_Seed': 1, 'Calc_Scenarios': 'No', 'Generate_Cashflows': 'Yes', 'Credit_Valuation_Adjustment': {'Calculate': 'Yes', 'First_To_Default_Adjustment': None, 'Counterparty': 'BNP Paribas (Paris)', 'Bank': 'Investec Limited', 'Deflate_Stochastically': 'Yes', 'Stochastic_Hazard_Rates': 'No', 'CDS_Tenors': '', 'Counterparty_Bank_Correlation': 0.0}, 'Collateral_Valuation_Adjustment': {'Calculate': 'No', 'Gradient': 'No'}, 'calc_name': 'CreditMonteCarlo.CrB_BNP_Paribas__Paris__ISDA', 'Object': 'CreditMonteCarlo'}


ERROR:root:Risk Factor InterestRate.USD-MASTER.USD-MASTER+240BP using stochastic process HullWhite2FactorImpliedInterestRateModel missing in Price Models section
ERROR:root:Risk Factor InterestRate.USD-MASTER.USD-MASTER+265BP using stochastic process HullWhite2FactorImpliedInterestRateModel missing in Price Models section
ERROR:root:Risk Factor InterestRate.USD-MASTER.USD-MASTER+235BP using stochastic process HullWhite2FactorImpliedInterestRateModel missing in Price Models section
ERROR:root:Risk Factor InterestRate.USD-MASTER.USD-MASTER+225BP using stochastic process HullWhite2FactorImpliedInterestRateModel missing in Price Models section
ERROR:root:Risk Factor InterestRate.ZAR-SWAP.ZAR-SWAP+115BP using stochastic process HullWhite2FactorImpliedInterestRateModel missing in Price Models section
ERROR:root:Risk Factor InterestRate.ZAR-SWAP.ZAR-SWAP+145BP using stochastic process HullWhite2FactorImpliedInterestRateModel missing in Price Models section
ERROR:root:Risk Factor InterestRate.

In [ ]:
%debug


In [ ]:
cp.data['CreditMonteCarlo.asdsad']

In [ ]:
#pp.tree_data
rfp.tree_data[-1]

In [ ]:
#rfp.right_container.children[0].children[4].plot.layout.max_width='600px'
dir(rfp.right_container.children[0].children[2])

In [ ]:
rfp.right_container.children[0].children[2].dropdown.layout.max_width='360px'
#rfp.right_container.children[0].children[2].selector.layout.max_width='600px'
rfp.right_container.children[0].children[2].selector